In [1]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import *
from technicals.patterns import apply_patterns
from technicals.patternsInRange import apply_patterns_in_range
from guruTester import GuruTester,GuruTester2
from plotting import CandlePlot
import MetaTrader5 as mt5
from datetime import datetime
from getCandle import Rates
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler  
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [2]:
def featureExtraction(df):
    candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
           'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
           'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']
    df = apply_patterns_in_range(df)
    for cp in candle_patterns:
        df[cp] = df[cp].astype(float)

    df = BollingerBandsNonLinear(df)
#     df = ATRFeature(df)
    df = KeltnerChannelsNonLinear(df)
    df = RSINonLinear(df)
    df = MACDNonLinear(df)
    df = BREAKOUT(df)
    df = ichimokuNonLinear(df)

    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df

In [30]:
BUY = 2
SELL = 0
NONE = 1

In [31]:
PROFIT_FACTOR = 1.2
def labeling(df):
    df['SIGNAL_LABEL'] = NONE
    df['TP'] = 0
    df['SL'] = 0
    
    for i in tqdm(range(1,len(df)) ):
        for j in range(i,len(df)):
            if( df.low[j] <= df.low[i-1] ):
                break
            else:
                if(df.high[j] > PROFIT_FACTOR*(df.close[i] - df.low[i-1] ) + df.close[i]):
                    df.SIGNAL_LABEL[i] = BUY
                    break


        for j in range(i,len(df)):
            if( df.high[j] >= df.high[i-1] ):
                break
            else:
                if(df.low[j] < PROFIT_FACTOR*(df.close[i] - df.high[i-1] ) + df.close[i]):
                    df.SIGNAL_LABEL[i] = SELL
                    break

        
    return df

In [32]:
pairs = ["GBPUSD","EURUSD","USDCHF","USDCAD","EURCHF"]
NB_H1_CANDLES = 20000

In [33]:
all_data = {}
for pair in pairs:
    rates = Rates(pair, NB_H1_CANDLES, mt5.TIMEFRAME_H1)
    SPREAD = rates.get_spread()
    df_an = rates.get_rates_from_now()
    df_an.drop(NB_H1_CANDLES-1,inplace=True) 
    df_an.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)
    all_data[pair] = df_an
    

In [34]:
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD # 0

0.00011

In [35]:
all_data_train = {}
all_data_test = {}
for pair in pairs:
    SLIT_INDEX = int(0.8*len(all_data[pair]))
    all_data_train[pair] = all_data[pair].iloc[:SLIT_INDEX]
    all_data_train[pair].reset_index(drop=True, inplace=True)
    all_data_test[pair] = all_data[pair].iloc[SLIT_INDEX:]
    all_data_test[pair].reset_index(drop=True, inplace=True)

In [36]:
all_data.keys()

dict_keys(['GBPUSD', 'EURUSD', 'USDCHF', 'USDCAD', 'EURCHF'])

In [37]:
# rates = Rates(pair, 13*len(df_test), mt5.TIMEFRAME_M5)
# df_m5 = rates.get_rates_from_now()
# df_m5.reset_index(drop=True, inplace=True)
# df_m5.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)

In [38]:
for pair in pairs:
    all_data_train[pair] = featureExtraction(all_data_train[pair])
    all_data_train[pair] = labeling(all_data_train[pair])
    all_data_test[pair] = featureExtraction(all_data_test[pair])
    all_data_test[pair] = labeling(all_data_test[pair])


C:\Users\meysam\Desktop\algotramic\technicals\indicators.py:215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['last_pivot_h'].iloc[i] = last_pivot_h
C:\Users\meysam\Desktop\algotramic\technicals\indicators.py:216: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['last_pivot_l'].iloc[i] = last_pivot_l
  0%|                                                                                        | 0/15917 [00:00<?, ?it/s]C:\Users\meysam\AppData\Local\Temp\ipykernel_11252\2628886360.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: ht

  0%|                                                                                         | 0/3921 [00:00<?, ?it/s]C:\Users\meysam\AppData\Local\Temp\ipykernel_11252\2628886360.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.SIGNAL_LABEL[i] = SELL
C:\Users\meysam\AppData\Local\Temp\ipykernel_11252\2628886360.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.SIGNAL_LABEL[i] = BUY
100%|█████████████████████████████████████████████████████████████████████████████| 3921/3921 [00:08<00:00, 437.63it/s]
C:\Users\meysam\Desktop\algotramic\technicals\indicators.py:215: SettingWithCopyWarning: 
A value is tryin

In [39]:

feature_col = [
    'BB_MA_cross', 'BB_UP_touch','BB_LW_touch',
    'EMA_cross', 'KeUp_touch', 'KeLo_touch', 
    'RSI_level',
    'MACDHIST_level',
#      'BREAK_UP', 'BREAK_DOWN',
        'Tenkan_sen_touch', 'Kijun_sen_touch',
       'Senkou_Span_A_touch', 'Senkou_Span_B_touch',
        'Ten_kij_cross','Cloud_cross',
#         'full_range',# this is added ...
#        'body_lower', 'body_upper', 
#         'body_bottom_perc', 'body_top_perc',
#        'body_perc', 'direction', 'body_size', 'low_change', 'high_change',
#     'mid_point', 'mid_point_prev_2', 'body_size_prev',
#        'direction_prev', 'direction_prev_2', 'body_perc_prev','body_perc_prev_2',
        'direction',
        'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING',
    'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR'
]

In [40]:
class RNN(nn.Module):
    def __init__(self, batch_size, input_size, hidden_dim,lstm_layers):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.batch_size = batch_size
        self.hidden_dim = hidden_dim
        self.LSTM_layers = lstm_layers
    
        self.dropout = nn.Dropout(0.5)
        self.rnn = nn.GRU(input_size=input_size, hidden_size=self.hidden_dim, num_layers=self.LSTM_layers, batch_first=True)
        self.fc1 = nn.Linear(in_features=self.hidden_dim, out_features=self.hidden_dim)
        self.fc2 = nn.Linear(in_features=self.hidden_dim, out_features=3)
#         self.fc2 = nn.Linear(257, 1)

    def forward(self, x):
        h = torch.zeros((self.LSTM_layers, x.size(0), self.hidden_dim))
#         c = torch.zeros((self.LSTM_layers, x.size(0), self.hidden_dim))
#         torch.nn.init.xavier_normal_(h)
#         torch.nn.init.xavier_normal_(c)
        out, _ = self.rnn(x, h)
#         out = self.dropout(out)
        out = torch.relu_(self.fc1(out[:,-1,:]) )
#         out = self.dropout(out)
        out = self.fc2(out)
        return out

In [41]:
class dataset(Dataset):
    def __init__(self,all_data,sequence_lenght,feature_col):
        self.all_data = all_data
        self.sequence_lenght = sequence_lenght
        self.feature_col = feature_col
        self.X = []
        self.Y = [] 
        self.indeces = []
        print("preparing data..")
        self.prepare_data()
        
    def prepare_data(self):
        for pair in self.all_data.keys():
            for i in tqdm(range(self.sequence_lenght+1,len(self.all_data[pair])) ):
    #             if self.df.pivots_h[i-1] or self.df.pivots_l[i-1]:
                X_feat = [] 
                for j in range(self.sequence_lenght):
                    feature = []
                    feature.append(self.all_data[pair].pivots_h[i-self.sequence_lenght+j])
                    feature.append(self.all_data[pair].pivots_l[i-self.sequence_lenght+j])
                    for fc in self.feature_col:
                        feature.append(self.all_data[pair][fc][i-self.sequence_lenght+j+1])

                    X_feat.append(feature)  
                self.X.append(np.array(X_feat))# TODO: use torch tensor 
                self.Y.append(self.all_data[pair].SIGNAL_LABEL[i])
                self.indeces.append(i)
                
    def get_feature_len(self):
        return self.X[0].shape[1]
    
    def __len__(self):
        return len(self.Y)
    
    def __getitem__(self,index):
        return torch.from_numpy(self.X[index]).float(),self.Y[index]

In [42]:
epoch = 1000
batch_size = 600
sequence_lenght = 20

In [43]:
ds_train = dataset(all_data_train,sequence_lenght,feature_col)
ds_test = dataset(all_data_test,sequence_lenght,feature_col) 

preparing data..


100%|███████████████████████████████████████████████████████████████████████████| 15881/15881 [02:11<00:00, 120.82it/s]


preparing data..


100%|█████████████████████████████████████████████████████████████████████████████| 3901/3901 [00:30<00:00, 127.26it/s]


In [44]:
loader_train = DataLoader(ds_train, batch_size=batch_size,shuffle=True)
loader_test = DataLoader(ds_test, batch_size=batch_size,shuffle=False)

In [45]:
print(len(loader_train))
print(len(loader_test))
print(len(ds_train))
print(len(ds_test))

133
33
79464
19505


In [46]:
ds_train.get_feature_len() 

26

In [47]:
input_size = ds_train.get_feature_len() 
hidden_dim = 60
lstm_layers = 2

In [48]:
model = RNN(batch_size, input_size, hidden_dim,lstm_layers)

In [49]:
weight = torch.Tensor([0.45,0.1,0.45])

In [50]:
criterion = nn.CrossEntropyLoss(weight=weight)
# criterion = nn.CrossEntropyLoss()

In [51]:
optimizer = optim.Adam(model.parameters(),lr=1e-3)

In [ ]:
model.train()
train_loss = []
test_loss = []
test_accuracy = []
for i in range(epoch):
    loss_epoch_train = 0
    for x_batch, y_batch in tqdm(loader_train):
        pred = model(x_batch)
        loss = criterion(pred,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_epoch_train += loss.detach().item()
        
    
    loss_epoch_test = 0
    model.eval()
    pred_test = []
    all_label = []
    
    for x_batch, y_batch in tqdm(loader_test):
        pred = model(x_batch)
        loss = criterion(pred,y_batch)
        loss_epoch_test += loss.detach().item()
        pred_test += list(pred.softmax(dim=1).argmax(dim=1).detach().numpy())
        all_label += list(y_batch.detach().numpy())
        
    train_loss.append(loss_epoch_train/len(loader_train))
    test_loss.append(loss_epoch_test/len(loader_test))
    pred_test = np.array(pred_test)
    all_label = np.array(all_label)
    conf = confusion_matrix(all_label, pred_test)
    acc_buy = conf[2,2]/conf[2,:].sum()
    acc_sell = conf[0,0]/conf[0,:].sum()
    print(conf)
    test_accuracy.append((pred_test==all_label).sum()/len(all_label))
    print(train_loss[-1],test_accuracy[-1],acc_buy,acc_sell)

100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [00:06<00:00,  5.48it/s]


[[4232    0  969]
 [4486    0 5112]
 [ 642    0 4064]]
0.8780608464004402 0.42532683927198156 0.8635784105397365 0.813689675062488


100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [00:07<00:00,  4.55it/s]


[[4459   59  683]
 [4594  445 4559]
 [ 798   51 3857]]
0.7253772750832981 0.4491668802871059 0.819592010199745 0.857335127860027


100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [00:06<00:00,  4.77it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [00:05<00:00,  5.91it/s]


[[2839 1706  656]
 [2378 4953 2267]
 [ 670 1633 2403]]
0.18581863565552503 0.5226864906434248 0.5106247343816405 0.5458565660449913


100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [00:05<00:00,  6.05it/s]


[[2737 1779  685]
 [2241 5048 2309]
 [ 623 1639 2444]]
0.1760162098291225 0.5244296334273263 0.5193370165745856 0.5262449528936743


 65%|████████████████████████████████████████████████████▉                            | 87/133 [00:32<00:15,  2.91it/s]

In [ ]:
df_test = df_an[['time', 'open', 'high', 'low', 'close','TP','SL','pivots_l','pivots_h']].iloc[indeces_test[0]:].copy()
df_test['SIGNAL'] = 0
for i,idx in enumerate(indeces_test):
    if i > 0:
#         df_test.SIGNAL.loc[idx] = y_perdict[i]
        if df_test.pivots_l[idx-1]==1 and y_perdict[i] ==BUY:
            df_test.SIGNAL.loc[idx] = y_perdict[i]-1
        if df_test.pivots_h[idx-1]==1 and y_perdict[i] ==SELL:
            df_test.SIGNAL.loc[idx] = y_perdict[i]-1

df_test.reset_index(drop=True, inplace=True)

In [ ]:
gt = GuruTester2(
        df_test,
        df_m5,
        SPREAD,
        use_spread=True
    )
df_res_m5 = gt.run_test()

In [ ]:
df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):
    if(df_res_m5.result[i] == -1):
        loss_ratio =1# abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
#         print(loss_ratio)
        df_res_m5['balance'][i] = perv_balance*(1-loss_ratio*loss_margin) 
    elif(df_res_m5.result[i] == 2):
        profit_to_loss = 2#abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
#         print(profit_to_loss)
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]

In [ ]:
df_res_m5_plot = df_res_m5#.loc[:100]
df_res_m5_plot['time'] = df_res_m5_plot['end_time']
cp = CandlePlot(df_res_m5_plot, candles=False)
print("min balance ",min(df_res_m5['balance']))
cp.show_plot(line_traces=['balance'])